In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
#     "unsloth/llama-2-7b-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit",
#     "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
#     "unsloth/gemma-2b-bnb-4bit",
#     "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
#     "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 8,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps = 200,
        max_steps = 100,
        learning_rate = 1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.605 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,1.767400
2,1.821300
3,1.870200
4,1.625600
5,1.880800
6,1.604400
7,1.679600
8,1.646500
9,1.743700
10,1.584200


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1701.9688 seconds used for training.
28.37 minutes used for training.
Peak reserved memory = 8.039 GB.
Peak reserved memory for training = 2.434 GB.
Peak reserved memory % of max memory = 54.509 %.
Peak reserved memory for training % of max memory = 16.504 %.


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
8.039 GB of memory reserved.


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\nThe next number in the Fibonacci sequence is 13.<|end_of_text|>']

In [ ]:
# Adding the custom prompts for generation
prompts = [
    ("Solve the following equation.", "x^2 - 4x + 4 = 0"),
    ("Explain the process of photosynthesis.", ""),
    ("Describe the differences between supervised and unsupervised machine learning.", ""),
    ("What caused World War I?", ""),
    ("Summarize the plot of 'Romeo and Juliet'.", ""),
    ("What are the Seven Wonders of the World?", ""),
    ("Write a Python function to check if a number is a palindrome.", "")
]

for instruction, input_text in prompts:
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                instruction,  # Instruction
                input_text,   # Input
                ""            # Output placeholder
            )
        ], return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
    print(f"Instruction: {instruction}")
    print(f"Input: {input_text}")
    print(f"Output: {tokenizer.batch_decode(outputs)[0]}")

FastLanguageModel.for_inference(model)
model.save_pretrained("lora_model")
print("Model successfully saved and uploaded!")

Instruction: Solve the following equation.
Input: x^2 - 4x + 4 = 0
Output: <|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Solve the following equation.

### Input:
x^2 - 4x + 4 = 0

### Response:
The solution to the equation is:
x = 2 or x = 2<|end_of_text|>
Instruction: Explain the process of photosynthesis.
Input: 
Output: <|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Explain the process of photosynthesis.

### Input:


### Response:
Photosynthesis is the process by which plants use energy from the sun to convert carbon dioxide and water into glucose (sugar) and oxygen. The process occurs in the chloroplasts of plant cells and involves the following steps:

1. Absorption of sunlight: Chlorophyll,

In [ ]:
!pip install datasets transformers evaluate nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00


In [ ]:
# Install required libraries
!pip install sacrebleu rouge-score

# Import necessary modules
from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer

# Function to compute BLEU score
def compute_bleu(references, candidates):
    bleu = corpus_bleu(candidates, [references])
    return bleu.score

# Function to compute ROUGE scores
def compute_rouge(references, candidates):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(ref, cand) for ref, cand in zip(references, candidates)]
    return rouge_scores

# Example input/output for evaluation
references = [
    "Photosynthesis is the process by which plants use sunlight to produce food.",
    "Supervised learning involves labeled data for training."
]
candidates = [
    "Plants make food using sunlight in photosynthesis.",
    "Labeled data is used in supervised learning."
]

# Evaluate BLEU
bleu_score = compute_bleu(references, candidates)
print(f"BLEU Score: {bleu_score:.2f}")

# Evaluate ROUGE
rouge_scores = compute_rouge(references, candidates)
for idx, scores in enumerate(rouge_scores):
    print(f"Example {idx + 1} ROUGE Scores: {scores}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=2c7249d2f497a9f86c11dc215e89d08474456b96adfca5961d64a06b67f8dd5d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
BLEU Score: 3.16
Example 1 ROUGE Scores: {'rouge1': Score(precision=0.7142857142857143, recall=0.4166666666666667, fmeasure=0.5263157894736842), 'rouge2': Score(precision=0.16666666666666666, recall=0.09090909090909091, fmeasure=0.11764705882352942), 'rougeL': Score(precision=0.42857142857142855, recall=0.25, fmeasure=0.3157894736842105)}
Example 2 ROUGE Scores: {'rouge1': Score(precision=0.5714285714285714, recall=0.5714285714285714, fmeasure=0.5714285714285714), 'rouge2': Score(precision=0.3333

In [ ]:
# Install required libraries
!pip install sacrebleu rouge-score

# Import necessary modules
from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer

# Function to compute BLEU score
def compute_bleu(references, candidates):
    bleu = corpus_bleu(candidates, [references])
    return bleu.score

# Function to compute ROUGE scores
def compute_rouge(references, candidates):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(ref, cand) for ref, cand in zip(references, candidates)]
    return rouge_scores

# Example input/output for evaluation
references = [
    "Photosynthesis is the process by which plants use sunlight to produce food.",
    "Supervised learning involves labeled data for training."
]
candidates = [
    "Plants make food using sunlight in photosynthesis.",
    "Labeled data is used in supervised learning."
]

# Evaluate BLEU
bleu_score = compute_bleu(references, candidates)
print(f"BLEU Score: {bleu_score:.2f}")

# Evaluate ROUGE
rouge_scores = compute_rouge(references, candidates)
for idx, scores in enumerate(rouge_scores):
    print(f"Example {idx + 1} ROUGE Scores: {scores}")


BLEU Score: 3.16
Example 1 ROUGE Scores: {'rouge1': Score(precision=0.7142857142857143, recall=0.4166666666666667, fmeasure=0.5263157894736842), 'rouge2': Score(precision=0.16666666666666666, recall=0.09090909090909091, fmeasure=0.11764705882352942), 'rougeL': Score(precision=0.42857142857142855, recall=0.25, fmeasure=0.3157894736842105)}
Example 2 ROUGE Scores: {'rouge1': Score(precision=0.5714285714285714, recall=0.5714285714285714, fmeasure=0.5714285714285714), 'rouge2': Score(precision=0.3333333333333333, recall=0.3333333333333333, fmeasure=0.3333333333333333), 'rougeL': Score(precision=0.2857142857142857, recall=0.2857142857142857, fmeasure=0.2857142857142857)}


In [ ]:
from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer

# Function to compute BLEU score
def compute_bleu(references, candidates):
    bleu = corpus_bleu(candidates, [references])
    return bleu.score

# Function to compute ROUGE scores
def compute_rouge(references, candidates):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(ref, cand) for ref, cand in zip(references, candidates)]
    return rouge_scores

# Test dataset (references and outputs)
references = [
    "For the 2 days he gets 2*3=6 hours\nHe gets 60% of the recommended amount for 7-2=5 days\nOn those days he gets 8*.6=4.8 hours\nSo he gets 5*4.8=24 hours\nSo he gets a total of 24+6=30 hours of sleep",
    "The first 4 customers bought 4*2=8 paintings\nThe next 12 customers bought 12*1=12 paintings\nThe last 4 customers bought 4*4=16 paintings\nIn total Tracy sold 8+12+16=36 paintings",
    "Cameron answered 2 * 6 = 12 questions for the first group.\nHe answered 2 * 11 = 22 questions for the second group.\nThe third group had 8 - 1 = 7 tourists that asked 2 questions each.\nThe third group also had 1 tourist that asked 2 * 3 = 6 questions.\nThus, the third group asked 7 * 2 + 6 = 14 + 6 = 20 questions.\nThe last group asked 7 * 2 = 14 questions.\nFor all four tours, Cameron answered 12 + 22 + 20 + 14 = 68 questions.",
    "The community pantry project gets 240 x 1/3 = $80.\nThe local crisis fund has 240 x 1/2 = $120.\nThe total funds allocated for the two funds is $80 + $120 = $200.\nSo, $240 - $200 = $40 are left for the other 2 funds.\nThe livelihood project gets $40 x 1/4 = $10.\nThis means that the contingency fund is $40 - $10 = $30.",
    "They gave money to 1000*.2=200 people\nIt cost 200*2000=$400,000\nThere was a 5 to 1 return so the government collected 5*400000=$2,000,000\nThat means the profit was 2,000,000-400,000=$1,600,000"
]

# Example generated outputs (candidates)
candidates = [
    "For two days, John slept 6 hours total. On the other five days, he slept 24 hours, totaling 30 hours of sleep for the week.",
    "Tracy sold 36 paintings at the art fair: 8 to the first 4 customers, 12 to the next 12 customers, and 16 to the last 4 customers.",
    "Cameron answered 68 questions today. The first group asked 12 questions, the second group asked 22, the third group asked 20, and the fourth group asked 14.",
    "Lyn donates $240 yearly. After allocations, $30 goes to the contingency fund.",
    "The government profited $1,600,000 from the stimulus program, with a return of 5 times the $400,000 investment."
]

# Compute BLEU Score
bleu_score = compute_bleu(references, candidates)
print(f"BLEU Score: {bleu_score:.2f}")

# Compute ROUGE Scores
rouge_scores = compute_rouge(references, candidates)
for idx, scores in enumerate(rouge_scores):
    print(f"Example {idx + 1} ROUGE Scores:")
    for metric, score in scores.items():
        print(f"  {metric}: {score.fmeasure:.2f}")



BLEU Score: 2.48
Example 1 ROUGE Scores:
  rouge1: 0.44
  rouge2: 0.19
  rougeL: 0.36
Example 2 ROUGE Scores:
  rouge1: 0.60
  rouge2: 0.36
  rougeL: 0.38
Example 3 ROUGE Scores:
  rouge1: 0.46
  rouge2: 0.23
  rougeL: 0.38
Example 4 ROUGE Scores:
  rouge1: 0.16
  rouge2: 0.06
  rougeL: 0.14
Example 5 ROUGE Scores:
  rouge1: 0.38
  rouge2: 0.13
  rougeL: 0.22
